# Zero-shot mutant prediction For VHH

In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import pandas as pd
from Bio import SeqIO
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")
import os
os.environ["http_proxy"] = "http://g5:15777"
os.environ["https_proxy"] = "http://g5:15777"

def read_seq(seq_file):
    for record in SeqIO.parse(seq_file, "fasta"):
        return str(record.seq)

wild_type = f"../example_data/expr/tgod_d4k.fasta"
sequence = read_seq(wild_type)

all_mutants = []
for i, wt in enumerate(sequence):
    for aa in "ACDEFGHIKLMNPQRSTVWY":
        if aa != wt:
            mutant = f"{wt}{i+1}{aa}"
            all_mutants.append(mutant)
df = pd.DataFrame({"mutant": all_mutants})

In [12]:
base_model_path = "AI4Protein/tgo_d4k_"

for i in range(0, 5):
    model_path = f"{base_model_path}{i}"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = AutoModelForMaskedLM.from_pretrained(model_path, trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained("AI4Protein/Prime_690M", trust_remote_code=True)
    model.eval()
    model = model.to(device)

    tokenized = tokenizer(sequence, return_tensors="pt").to(device)
    input_ids = tokenized["input_ids"]
    attention_mask = tokenized["attention_mask"]

    with torch.no_grad():
        output = model(input_ids, attention_mask=attention_mask)
        logits = output.logits[0, 1:-1, :].log_softmax(dim=-1)
    
    scores = []
    for mutant in tqdm(df["mutant"]):
        score = 0
        for sub_mutant in mutant.split(":"):
            wt, idx, mt = sub_mutant[0], int(sub_mutant[1:-1]) - 1, sub_mutant[-1]
            score += (logits[idx, tokenizer.get_vocab()[mt]] - logits[idx, tokenizer.get_vocab()[wt]]).item()
        scores.append(score)
    df[f"model_{i}"] = scores
    print(f"Model {i} done")

In [ ]:
aggregate_score = 0
for i in range(0, 5):
    # min-max
    df[f"model_{i}"] = (df[f"model_{i}"] - df[f"model_{i}"].min()) / (df[f"model_{i}"].max() - df[f"model_{i}"].min())
    aggregate_score += df[f"model_{i}"]
df["aggregate"] = aggregate_score
df = df.sort_values("aggregate", ascending=False)

In [ ]:
df.head(50)

,mutant,model_1,model_2,model_3,model_4,model_5,aggregate
2431,P128Y,1.000000,0.970255,0.443269,0.444688,0.874523,3.732735
3400,P179Y,0.990596,0.972601,0.443834,0.448956,0.875515,3.731502
2963,P156Y,0.990545,0.972510,0.442669,0.445068,0.872513,3.723305
2184,P115Y,0.993481,0.959609,0.444739,0.445053,0.877184,3.720067
1500,P79Y,0.988734,0.969609,0.446817,0.446733,0.867236,3.719127
1709,P90Y,0.991918,0.963270,0.451155,0.446952,0.863504,3.716798
1975,P104Y,0.998938,0.964343,0.441291,0.444411,0.866725,3.715709
4977,P262Y,0.991169,0.978409,0.441124,0.431759,0.872446,3.714907
4540,P239Y,0.991518,0.979126,0.442047,0.432623,0.869247,3.714560
3856,P203Y,0.993159,0.973205,0.436474,0.442433,0.867401,3.712672
